In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, avg, count, min, max, row_number, lead, lag, first, last


spark = SparkSession.builder.appName("Zadanie3").getOrCreate()

In [0]:
transactions_data = [
    (1, '2011-01-01', 500), (1, '2011-01-15', 50), (1, '2011-01-22', 250),
    (1, '2011-01-24', 75), (1, '2011-01-26', 125), (1, '2011-01-28', 175),
    (2, '2011-01-01', 500), (2, '2011-01-15', 50), (2, '2011-01-22', 25),
    (2, '2011-01-23', 125), (2, '2011-01-26', 200), (2, '2011-01-29', 250),
    (3, '2011-01-01', 500), (3, '2011-01-15', 50), (3, '2011-01-22', 5000),
    (3, '2011-01-25', 550), (3, '2011-01-27', 95), (3, '2011-01-30', 2500)
]

transactions_df = spark.createDataFrame(transactions_data, ["AccountId", "TranDate", "TranAmt"])


window = Window.partitionBy("AccountId").orderBy("TranDate")
window_r = window.rowsBetween(-2, 0)
display(transactions_df)

AccountId,TranDate,TranAmt
1,2011-01-01,500
1,2011-01-15,50
1,2011-01-22,250
1,2011-01-24,75
1,2011-01-26,125
1,2011-01-28,175
2,2011-01-01,500
2,2011-01-15,50
2,2011-01-22,25
2,2011-01-23,125


In [0]:
transactions_df = transactions_df.withColumn("RunTotalAmt", sum("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("RunAvg", avg("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("RunTranQty", count("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("RunSmallAmt", min("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("RunLargeAmt", max("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("SlideAvg", avg("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("SlideQty", count("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("SlideMin", min("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("SlideMax", max("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("SlideTotal", sum("TranAmt").over(window_r))


transactions_df = transactions_df.withColumn("LeadValue", lead("TranAmt").over(window))
transactions_df = transactions_df.withColumn("LagValue", lag("TranAmt").over(window))
transactions_df = transactions_df.withColumn("FirstValue", first("TranAmt").over(window))
transactions_df = transactions_df.withColumn("LastValue", last("TranAmt").over(window_r))
transactions_df = transactions_df.withColumn("RowNumber", row_number().over(window))

display(transactions_df)


AccountId,TranDate,TranAmt,RunTotalAmt,RunAvg,RunTranQty,RunSmallAmt,RunLargeAmt,SlideAvg,SlideQty,SlideMin,SlideMax,SlideTotal,LeadValue,LagValue,FirstValue,LastValue,RowNumber
1,2011-01-01,500,500,500.0,1,500,500,500.0,1,500,500,500,50,null,500,500,1
1,2011-01-15,50,550,275.0,2,50,500,275.0,2,50,500,550,250,500,500,50,2
1,2011-01-22,250,800,266.6666666666667,3,50,500,266.6666666666667,3,50,500,800,75,50,500,250,3
1,2011-01-24,75,375,125.0,3,50,250,125.0,3,50,250,375,125,250,500,75,4
1,2011-01-26,125,450,150.0,3,75,250,150.0,3,75,250,450,175,75,500,125,5
1,2011-01-28,175,375,125.0,3,75,175,125.0,3,75,175,375,null,125,500,175,6
2,2011-01-01,500,500,500.0,1,500,500,500.0,1,500,500,500,50,null,500,500,1
2,2011-01-15,50,550,275.0,2,50,500,275.0,2,50,500,550,25,500,500,50,2
2,2011-01-22,25,575,191.66666666666666,3,25,500,191.66666666666666,3,25,500,575,125,50,500,25,3
2,2011-01-23,125,200,66.66666666666667,3,25,125,66.66666666666667,3,25,125,200,200,25,500,125,4
